# Pytorch based on official doc Introduction to Pytorch 

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [2]:
training_data = datasets.FashionMNIST(root="data", train=True, 
    download=True, transform=ToTensor(), )

test_data = datasets.FashionMNIST(root="data", train=False,
    download=True, transform=ToTensor(), )

In [3]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512), nn.ReLU(), nn.Linear(512, 512), nn.ReLU(), nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y), in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch%100 == 0:
            loss, current = loss.item(), batch*len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)

            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [10]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("done")

Epoch 1
-------------------------------
loss: 2.312259  [    0/60000]
loss: 2.296057  [ 6400/60000]
loss: 2.275776  [12800/60000]
loss: 2.269691  [19200/60000]
loss: 2.254359  [25600/60000]
loss: 2.219709  [32000/60000]
loss: 2.235388  [38400/60000]
loss: 2.191299  [44800/60000]
loss: 2.194493  [51200/60000]
loss: 2.163822  [57600/60000]
Test Error: 
 Accuracy: 47.8%, Avg loss: 2.156424 

Epoch 2
-------------------------------
loss: 2.170651  [    0/60000]
loss: 2.159404  [ 6400/60000]
loss: 2.097608  [12800/60000]
loss: 2.119868  [19200/60000]
loss: 2.065700  [25600/60000]
loss: 1.996933  [32000/60000]
loss: 2.041130  [38400/60000]
loss: 1.943093  [44800/60000]
loss: 1.959476  [51200/60000]
loss: 1.892565  [57600/60000]
Test Error: 
 Accuracy: 54.9%, Avg loss: 1.884977 

Epoch 3
-------------------------------
loss: 1.918921  [    0/60000]
loss: 1.891506  [ 6400/60000]
loss: 1.765785  [12800/60000]
loss: 1.820157  [19200/60000]
loss: 1.703855  [25600/60000]
loss: 1.639918  [32000/600

In [11]:
torch.save(model.state_dict(), "model1.pth")
print("saved to model1.pth")

saved to model1.pth


In [12]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model1.pth"))

<All keys matched successfully>

In [16]:
classes = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker",
    "Bag", "Ankle boot", ]
model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


## Tensors

In [17]:
import torch
import numpy as np

In [20]:
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)
print(x_data)

tensor([[1, 2],
        [3, 4]])


In [22]:
x_np = torch.from_numpy(np.array(data))
print(x_np)

tensor([[1, 2],
        [3, 4]])
